In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import time
import numpy as np

import nltk
from nltk.stem.porter import *
import string
from nltk.tokenize import TweetTokenizer
token = TweetTokenizer()
stemmer = PorterStemmer() 
from nltk.corpus import stopwords # Import the stop word list

import time

## Load the dataset

In [2]:
# Take one of the books, with approx uniform distribution for its ratings
data = pd.read_csv('Paula_Hawkins-The-Girl-On-The-Train.csv', sep = '\t', header = None)

# Rename columns
data.columns = ['rating', 'tail', 'title', 'review']

## Cleaning

In [3]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review, "html.parser").get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [4]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
# nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [6]:
start = time.time()

sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in data["review"]:
    sentences += review_to_sentences(review, tokenizer)
      
print(time.time() - start)

Parsing sentences from training set


C:\Users\Peter martigny\Anaconda3\envs\py35\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
C:\Users\Peter martigny\Anaconda3\envs\py35\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'..'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


16.410303831100464


C:\Users\Peter martigny\Anaconda3\envs\py35\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.. .'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


## Build the model

In [7]:
start = time.time()

# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)


# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

print(time.time() - start)

C:\Users\Peter martigny\Anaconda3\envs\py35\lib\site-packages\gensim\utils.py:843: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-01-13 23:53:33,648 : INFO : 'pattern' package not found; tag filters are not available for English
2017-01-13 23:53:33,654 : INFO : collecting all words and their counts
2017-01-13 23:53:33,655 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-01-13 23:53:33,699 : INFO : PROGRESS: at sentence #10000, processed 188196 words, keeping 8922 word types
2017-01-13 23:53:33,737 : INFO : PROGRESS: at sentence #20000, processed 355153 words, keeping 11357 word types
2017-01-13 23:53:33,771 : INFO : PROGRESS: at sentence #30000, processed 498166 words, keeping 12705 word types
2017-01-13 23:53:33,801 : INFO : PROGRESS: at sentence #40000, processed 630564 words, keeping 13729 word types
2017-01-13 23:53:33,830 : INFO : PROGRESS: at sentence

Training model...


2017-01-13 23:53:33,855 : INFO : PROGRESS: at sentence #60000, processed 836923 words, keeping 14920 word types
2017-01-13 23:53:33,883 : INFO : PROGRESS: at sentence #70000, processed 924516 words, keeping 15254 word types
2017-01-13 23:53:33,904 : INFO : PROGRESS: at sentence #80000, processed 1002926 words, keeping 15658 word types
2017-01-13 23:53:33,923 : INFO : PROGRESS: at sentence #90000, processed 1065784 words, keeping 16033 word types
2017-01-13 23:53:33,944 : INFO : collected 16495 word types from a corpus of 1128993 raw words and 99982 sentences
2017-01-13 23:53:33,946 : INFO : Loading a fresh vocabulary
2017-01-13 23:53:33,962 : INFO : min_count=40 retains 1522 unique words (9% of original 16495, drops 14973)
2017-01-13 23:53:33,963 : INFO : min_count=40 leaves 1061819 word corpus (94% of original 1128993, drops 67174)
2017-01-13 23:53:33,971 : INFO : deleting the raw counts dictionary of 16495 items
2017-01-13 23:53:33,974 : INFO : sample=0.001 downsamples 73 most-common

5.300626993179321
